# Titanic (Kaggle) — Modèle logistique pour interface interactive

Objectif :
- Charger et préparer le dataset Titanic (`train.csv`)
- Nettoyer et créer les variables nécessaires
- Entraîner un modèle de régression logistique
- Extraire :
    - intercept
    - coefficients
    - catégories de référence
- Exporter le tout dans un fichier `titanic_model.json` exploitable côté frontend


## 00. Imports

In [1]:
import pandas as pd
import numpy as np
import json

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

## 01. Chargement des données

On charge `train.csv` (fichier Kaggle original).  
Il contient :
- 891 passagers
- Variable cible `Survived` (0 = non, 1 = oui)


In [2]:
df = pd.read_csv("data/train.csv", sep=None, engine="python")
print(df.shape)
df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 02. Exploration rapide

On vérifie :
- La taille du dataset
- Les types de variables
- Les valeurs manquantes


In [3]:
print("Types de données :")
df.dtypes

Types de données :


PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [4]:
print("Valeurs manquantes :")
df.isnull().sum().sort_values(ascending=False)

Valeurs manquantes :


Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

In [5]:
print("Pclass :", sorted(df['Pclass'].dropna().unique()))
print("Sex :", df['Sex'].dropna().unique().tolist())
print("Embarked :", df['Embarked'].dropna().unique().tolist())
print("Survived :", sorted(df['Survived'].unique()))

Pclass : [1, 2, 3]
Sex : ['male', 'female']
Embarked : ['S', 'C', 'Q']
Survived : [0, 1]


## 03. Nettoyage & Feature Engineering

Objectifs :
1) **Imputer** les valeurs manquantes :
   - `Embarked` : imputé avec la **valeur la plus fréquente** via `SimpleImputer(strategy="most_frequent")`.  
     → Évite de supprimer des lignes tout en restant cohérent statistiquement (S est souvent majoritaire).
   - `Age` : imputé par **médiane** (robuste aux valeurs extrêmes).

2) **Créer de nouvelles variables** utiles pour l’analyse et l’app interacive :
   - `age_group` : `child` si `Age < 15`, sinon `adult`.
   - `family_size` = `SibSp + Parch`. (SibSp = 'Siblings/Spouse, Parch = 'Parent/Children )
   - `family` : `with_family` si `family_size > 0`, sinon `alone`.



In [6]:
# Imputation Embarked (most_frequent) ---
imp_embarked = SimpleImputer(strategy="most_frequent")
df[["Embarked"]] = imp_embarked.fit_transform(df[["Embarked"]])

# Imputation Age (median) ---
imp_age = SimpleImputer(strategy="median")
df[["Age"]] = imp_age.fit_transform(df[["Age"]])

# Variables dérivées ---
df["age_group"] = np.where(df["Age"] < 15, "child", "adult")
df["family_size"] = df["SibSp"] + df["Parch"]
df["family"] = np.where(df["family_size"] > 0, "with_family", "alone")

In [7]:
df.isnull().sum().sort_values(ascending=False).head(10)

Cabin          687
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
dtype: int64

In [8]:
df[["Age", "age_group", "SibSp", "Parch", "family_size", "family"]].head()

,Age,age_group,SibSp,Parch,family_size,family
0,22.0,adult,1,0,1,with_family
1,38.0,adult,1,0,1,with_family
2,26.0,adult,0,0,0,alone
3,35.0,adult,1,0,1,with_family
4,35.0,adult,0,0,0,alone


## 04. Régression logistique

Objectif : estimer l'effet de chaque variable **en contrôlant les autres**.
Méthode :
- Encodage One-Hot des variables catégorielles (`drop='first'` pour définir une référence).
- Modèle `LogisticRegression`.
- On récupère l'**intercept** (niveau de base) et les **coefficients** (effets log-odds).

Interprétation rapide :
- Coefficient **positif** → augmente les chances de survie vs. la feature de référence.
- Coefficient **négatif** → diminue les chances de survie vs. la feature de référence.
- L’intercept correspond au log-odds pour l'observation **de référence** (toutes variables à leur catégorie de base).


In [9]:
X = df[['Sex', 'age_group', 'Pclass', 'Embarked', 'family']]
y = df['Survived']

encoder = OneHotEncoder(drop='first', sparse_output=True)
X_encoded = encoder.fit_transform(X)

model = LogisticRegression(max_iter=2000)
model.fit(X_encoded, y)

feature_names = encoder.get_feature_names_out(X.columns)
coef_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': model.coef_[0]
}).sort_values('coefficient', ascending=False).reset_index(drop=True)

intercept = float(model.intercept_[0])

print("Intercept (log-odds de la feature de référence) :", intercept)
coef_df.head(10)


Intercept (log-odds de la feature de référence) : 2.5033040563496187


,feature,coefficient
0,age_group_child,1.112857
1,Embarked_Q,-0.046910
2,family_with_family,-0.103998
3,Embarked_S,-0.573678
4,Pclass_2,-0.661571
5,Pclass_3,-1.866601
6,Sex_male,-2.505539


### Interprétation

**Intercept** (log-odds pour le profil de référence) : `2.5033`

Le profil de référence correspond à :  
- **Sex** : female  
- **age_group** : adult  
- **Pclass** : 1  
- **Embarked** : C  
- **family** : alone  

#### Cela signifie que, pour ce profil, les *log-odds* de survie sont de **2.5033**.  

##### Coefficients des autres features

| Feature              | Coefficient | Interprétation 
|----------------------|-------------|---
| age_group_child      | +1.1129     | Être un enfant augmente les log-odds de survie par rapport à un adulte (à caractéristiques identiques). 
| Embarked_Q           | -0.0469     | Embarquer à Queenstown a un effet légèrement négatif par rapport à Cherbourg (référence). 
| family_with_family   | -0.1040     | Voyager avec de la famille réduit légèrement les log-odds par rapport à voyager seul(e). 
| Embarked_S           | -0.5737     | Embarquer à Southampton réduit les log-odds par rapport à Cherbourg. 
| Pclass_2             | -0.6616     | Voyager en 2ᵉ classe réduit les log-odds par rapport à la 1ère classe. 
| Pclass_3             | -1.8666     | Voyager en 3ᵉ classe réduit fortement les log-odds par rapport à la 1ère classe. 
| Sex_male             | -2.5055     | Être un homme réduit fortement les log-odds par rapport à une femme. 


In [10]:
# Exemple profil de référence
profil_df = pd.DataFrame([{
    'Sex': 'female',
    'age_group': 'adult',
    'Pclass': 1,
    'Embarked': 'C',
    'family': 'alone'
}])

# Encodage avec le même encoder que pour l'entraînement
profil_encoded = encoder.transform(profil_df)

# Calcul de la probabilité avec notre modèle
prob = model.predict_proba(profil_encoded)[0, 1]  # proba de survie (classe 1)

print(f"Probabilité de survie : {prob:.2%}")


Probabilité de survie : 92.44%


In [11]:
# Coefficients avec noms des features
feature_names = encoder.get_feature_names_out(X.columns)
coefficients = dict(zip(feature_names, model.coef_[0]))

# Catégories de référence
categories_reference = {
    col: encoder.categories_[i][0]  # première catégorie = référence
    for i, col in enumerate(X.columns)
}

In [12]:
print(categories_reference)
print(coefficients)
print(feature_names)

{'Sex': 'female', 'age_group': 'adult', 'Pclass': 1, 'Embarked': 'C', 'family': 'alone'}
{'Sex_male': -2.5055390571245795, 'age_group_child': 1.112857059953048, 'Pclass_2': -0.6615708135867034, 'Pclass_3': -1.8666012088433828, 'Embarked_Q': -0.046909638561932365, 'Embarked_S': -0.5736781250321217, 'family_with_family': -0.1039976356144329}
['Sex_male' 'age_group_child' 'Pclass_2' 'Pclass_3' 'Embarked_Q'
 'Embarked_S' 'family_with_family']


## 05. Export JSON

Objectif : produire `titanic_model.json` prêt pour le composant `LogitPredictor`.

Le JSON contiendra :
- `intercept` (float)
- `coefficients` : map `{ "Var_valeur": coefficient }` (float)
- `schema.variables` : pour chaque variable catégorielle, son nom, la liste **complète** des catégories et la **référence** (1ère catégorie car `drop='first'`).

La cellule suivante :
1) S’assure que `X`, `encoder`, `model` existent (sinon les (ré)entraîne).
2) Extrait `intercept`, `coefficients`, `schema` depuis `OneHotEncoder`.
3) Convertit les types NumPy en types Python natifs.
4) Écrit `titanic_model.json`.


In [ ]:
# schema variables
variables = []
for name, cats in zip(X.columns, encoder.categories_):
    cats_py = []
    for c in cats:
        # cast numpy -> python natif
        if isinstance(c, (np.integer,)):
            cats_py.append(int(c))
        elif isinstance(c, (np.floating,)):
            cats_py.append(float(c))
        else:
            cats_py.append(str(c))
    variables.append({
        "name": name,
        "type": "categorical",
        "categories": cats_py,
        "reference": cats_py[0]  # 1ère catégorie = référence car drop='first'
    })

    titanic_model = {
    "intercept": intercept,
    "coefficients": coefficients,
    "schema": { "variables": variables }
}

with open("titanic_model.json", "w", encoding="utf-8") as f:
    json.dump(titanic_model, f, indent=2, ensure_ascii=False)


# Onglet Exploration

**Objectif :**
- Produire :
    1. Des statistiques "marginales" (influence d'une variable isolée sur la survie)
    2. Des statistiques "combinées" (probabilité de survie pour un profil complet)
- Exporter un fichier `titanic_stats.json` exploitable dans une application React

## 01. Marginales : probabilité de survie par variable isolée

Mesure de l'effet d'une seule variable sur la survie, sans tenir compte des autres variables.

Exemple : 
- Marginale `sex` = % de survie chez les hommes vs chez les femmes.
- Marginale `pclass` = % de survie pour chaque classe (1, 2, 3).

On calcule :
- `survival_rate` = proportion de passagers ayant survécu
- `count` = effectif du groupe


In [14]:
def survival_stats(groupby_col):
    return (
        df.groupby(groupby_col)
          .agg(survival_rate=('Survived', 'mean'), # proba moyenne de survie pour chaque groupe
               count=('Survived', 'size')) # effectif de chaque groupe
          .reset_index()
          .to_dict(orient='records') # df to dict --> pour l'intégration dans l'app
    )

marginals = {
    'sex': survival_stats('Sex'),
    'age_group': survival_stats('age_group'),
    'pclass': survival_stats('Pclass'),
    'embarked': survival_stats('Embarked'),
    'family': survival_stats('family')
}

for key, val in marginals.items():
    print(f"\n {key}")
    for row in val:
        print(row)



 sex
{'Sex': 'female', 'survival_rate': 0.7420382165605095, 'count': 314}
{'Sex': 'male', 'survival_rate': 0.18890814558058924, 'count': 577}

 age_group
{'age_group': 'adult', 'survival_rate': 0.36531365313653136, 'count': 813}
{'age_group': 'child', 'survival_rate': 0.5769230769230769, 'count': 78}

 pclass
{'Pclass': 1, 'survival_rate': 0.6296296296296297, 'count': 216}
{'Pclass': 2, 'survival_rate': 0.47282608695652173, 'count': 184}
{'Pclass': 3, 'survival_rate': 0.24236252545824846, 'count': 491}

 embarked
{'Embarked': 'C', 'survival_rate': 0.5535714285714286, 'count': 168}
{'Embarked': 'Q', 'survival_rate': 0.38961038961038963, 'count': 77}
{'Embarked': 'S', 'survival_rate': 0.33900928792569657, 'count': 646}

 family
{'family': 'alone', 'survival_rate': 0.30353817504655495, 'count': 537}
{'family': 'with_family', 'survival_rate': 0.5056497175141242, 'count': 354}


## 02. Combinaisons complètes : probabilité de survie par profil

Ici on calcule le taux de survie pour **chaque combinaison** de :
`Sex`, `age_group`, `Pclass`, `Embarked`, `family`.

⚠️ Attention aux petits effectifs : certaines combinaisons peuvent avoir très peu de lignes.
On garde la **taille du groupe** (`count`) pour pouvoir filtrer ensuite côté frontend
(ex. masquer ou annoter les combinaisons avec `count < 5`).


In [15]:
combo = (
    df.groupby(['Sex', 'age_group', 'Pclass', 'Embarked', 'family'])
      .agg(survival_rate=('Survived', 'mean'),
           count=('Survived', 'size'))
      .reset_index()
      .sort_values(['Sex', 'age_group', 'Pclass', 'Embarked', 'family'])
      .to_dict(orient='records')
)

# aperçu
combo[:3], len(combo)


([{'Sex': 'female',
   'age_group': 'adult',
   'Pclass': 1,
   'Embarked': 'C',
   'family': 'alone',
   'survival_rate': 0.9444444444444444,
   'count': 18},
  {'Sex': 'female',
   'age_group': 'adult',
   'Pclass': 1,
   'Embarked': 'C',
   'family': 'with_family',
   'survival_rate': 1.0,
   'count': 25},
  {'Sex': 'female',
   'age_group': 'adult',
   'Pclass': 1,
   'Embarked': 'Q',
   'family': 'with_family',
   'survival_rate': 1.0,
   'count': 1}],
 46)

## 03. Export JSON

On exporte :
- `marginals` (par variable isolée)  
- `combo` (profils complets)


In [16]:
stats = {
    'combo': combo,
    'marginals': marginals
}

with open('titanic_stats.json', 'w') as f:
    json.dump(stats, f, indent=2)
